In [ ]:
# name_input = "PKS 1156+295"
# name_input = "Mrk 421"
# name_input = "Crab"
# name_input = "PKS 2155-304"
# name_input = "QSO B0229+200"
# name_input = 'NGC 1068'
name_input = ""
ra_s = 166.113808  # ra in degree
dec_s = 38.208833 # dec in degree
image_size = 3. # in arcmin (integrate flux of all sources within this radius)
radius_photometry = 1. # in arcsec
dr=9 # data release
image_band='g' #
pixsize=1. # arcsec per pixel

In [ ]:
from astropy.io import ascii
import numpy as np
from numpy import pi, cos,sin
import os

# astroquery API that will be traced by Renku plugin
from astroquery.ned import Ned
from astroquery.simbad import Simbad 
Simbad.add_votable_fields('otype')


# Conventional astronomical tools, also to be traced by Renku plugin, there is domain-specific ontology built in
from astropy.wcs import WCS
import pyvo as vo
from astropy.io import fits
from astropy.coordinates import SkyCoord  # High-level coordinates
from astropy.coordinates import Angle, Latitude, Longitude  # Angles
import astropy.units as u
from astropy.constants import hbar
from astropy.time import Time

# conventional python routines
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from matplotlib.colors import LogNorm

nano_maggies_to_Jy = 3.631e-6 # constant of conversion
if not(os.path.isdir('figs')):
    os.makedirs('figs')
if not(os.path.isdir('data')):
    os.makedirs('data')

from astropy.table import Table
import hashlib
import base64
from astroquery.desi import DESILegacySurvey

In [ ]:
radius_photometry = Angle(radius_photometry * u.arcsec)
image_size = Angle(image_size * u.arcmin)
pixsize = Angle(pixsize * u.arcsec)

In [ ]:
if name_input == "":
    source = SkyCoord(ra_s,dec_s,unit='degree')
    result_simbad=Simbad.query_region(source, radius=radius_photometry)
    ra_simbad=result_simbad['RA'].data[0]
    dec_simbad=result_simbad['DEC'].data[0]
    name_simbad=result_simbad['MAIN_ID'].data[0]
    otype_simbad = result_simbad['OTYPE'].data[0]

    print('"'+ra_simbad+' '+dec_simbad+'"',name_simbad)
    coord_simbad=SkyCoord(ra_simbad+' '+dec_simbad, unit=(u.hour, u.deg))
    sep=source.separation(coord_simbad).arcsec
    if(sep<=radius_photometry.arcsec):
        name_input=name_simbad
    names=Simbad.query_objectids(name_input)
else:
    result_simbad = Simbad.query_object(name_input, wildcard=True)
    names=Simbad.query_objectids(name_input)
    ra_simbad = result_simbad['RA'].data[0] # here it is a string!
    dec_simbad = result_simbad['DEC'].data[0]
    otype_simbad = result_simbad['OTYPE'].data[0]
    name_simbad=result_simbad['MAIN_ID'].data[0]
    ra_s=Angle(ra_simbad,unit='hourangle').degree
    dec_s=Angle(dec_simbad,unit='deg').degree

npix=int(2*image_size/pixsize)
print(name_input, otype_simbad)
source = SkyCoord(ra_s,dec_s,unit='degree')
print(source)

In [ ]:
query=DESILegacySurvey.get_images(coordinates='icrs', radius=image_size, pixels=npix, survey='dr%d'%dr, position=source)
hdul = query[0]
hdul[0].header

In [ ]:
paramstring='ra='+str(ra_s)+'&dec='+str(dec_s)+'&size='+str(npix)+'&layer=ls-dr'+str(dr)+'&pixscale='+str(pixsize)+'&bands='+image_band
suffix = hashlib.md5(paramstring.encode()).hexdigest()
filename='data/image_legacysurvey_%s.fits'%( suffix )

if os.path.exists(filename):
        os.remove(filename)
hdul.writeto(filename)

In [ ]:
hdu = hdul[0]
image=hdu.data
wcs=WCS(hdu.header)
w = WCS(hdu.header)
sky = w.pixel_to_world(0, 0)
ra_max_image=sky.ra.degree
dec_min_image=sky.dec.degree
sky = w.pixel_to_world(npix-1, npix-1)
ra_min_image=sky.ra.degree
dec_max_image=sky.dec.degree

In [ ]:
query = DESILegacySurvey.query_region(coordinates=source, radius=image_size, data_release=dr)

tap_result = query
ra=tap_result['ra']
dec=tap_result['dec']
t=tap_result['type']
nsources=len(tap_result)
coord = SkyCoord(ra=ra, dec=dec, unit='deg',frame='icrs')
sep=source.separation(coord)
indexes=[]
for i in range(nsources):
    if(t[i]=='DUP'):
        sep[i]=100. * u.deg
        # TODO: this is just to ignore it from table, rather just delete from it
    if(sep[i]< radius_photometry): # FIXME: parameter here (or maybe not)
        indexes.append(i)
minsep=np.min(sep)
index=np.argmin(sep)
print('found LegacySurvey source number',indexes,'at separation',sep[indexes],'arcsec')
print(minsep,index)
ra_min=min(ra)
dec_min=min(dec)
ra_max=max(ra)
dec_max=max(dec)
print(min(ra),max(ra),min(dec),max(dec))
print(ra_min_image,ra_max_image,dec_min_image,dec_max_image)

tap_result.write('data/catalog_legacysurvey_%s.ecsv'%suffix, overwrite=True)

In [ ]:
plt.figure(figsize=(17,13))
im=plt.imshow(image, norm=LogNorm(vmax=np.max(image),vmin=np.max(image)/1e3), origin='lower',extent=(ra_max_image,ra_min_image,dec_min_image,dec_max_image))
plt.grid(color='black', ls='solid')
plt.scatter(ra,dec,color='red',alpha=0.9)
plt.scatter([ra_s],[dec_s],color='blue',linewidth=4,alpha=0.3)

plt.xlabel('RA',fontsize=16)
plt.ylabel('DEC',fontsize=16)

plt.tick_params(axis='both', which='major', labelsize=16)
plt.colorbar(im)


plt.savefig('figs/image_legacysurvey_%s_log.png'%( suffix ))

In [ ]:
plt.figure(figsize=(17,13))
im=plt.imshow(image, vmax=np.max(image),vmin=np.max(image)/1e3, origin='lower',extent=(ra_max_image,ra_min_image,dec_min_image,dec_max_image))
plt.grid(color='black', ls='solid')
#plt.scatter(ra,dec,color='white',alpha=0.3)
#plt.scatter([ra_degree],[dec_degree],color='blue',linewidth=4,alpha=0.3)

plt.xlabel('RA',fontsize=16)
plt.ylabel('DEC',fontsize=16)

plt.tick_params(axis='both', which='major', labelsize=16)
plt.colorbar(im)

plt.savefig('figs/image_legacysuvery_linear.png')

In [ ]:
label=['g','r','z','w1','w2','w3','w4']
wavelength=np.array([4770,6231,9134,3.368e4,4.618e4,12.082e4,22.194e4])
wavelength=wavelength*1e-8 # in cm
frequency=3.e10/wavelength # in Hz
energy=2*pi*hbar.to('eV s')/u.eV/u.s*frequency # in eV
factor=1e-23*frequency # conversion of Jy to erg/cm2s
flux=np.zeros(len(label))
err=np.zeros(len(label))
for i in range(len(label)):
    for j in range(len(indexes)):
        if (tap_result['flux_ivar_'+label[i]][indexes[j]]>0):
            flux[i]+=tap_result['flux_'+label[i]][indexes[j]]
            err[i]+=1 / tap_result['flux_ivar_'+label[i]][indexes[j]]

    flux[i]=flux[i]/tap_result['mw_transmission_'+label[i]][index]*nano_maggies_to_Jy*factor[i]
    err[i]=np.sqrt(err[i])/tap_result['mw_transmission_'+label[i]][index]*nano_maggies_to_Jy*factor[i]
    print(energy[i],flux[i],err[i])

data = Table()
data['E'] = energy*u.eV
data['nuFnu'] = flux*u.erg/u.cm**2/u.s
data['nuFnu_err'] = err*u.erg/u.cm**2/u.s
data.write('data/spectrum_legacysurvey_%s.ecsv'%suffix, overwrite=True)
data.info()

In [ ]:
plt.figure(figsize=(10,7))
plt.errorbar(energy, flux,err, fmt='o-', color='red', linewidth=1)

plt.xlabel('$E$, [eV]',fontsize=16)
plt.ylabel('$E^2dN/dE$, [erg/(cm$^2$s)]',fontsize=16)
plt.xscale('log')
plt.yscale('log')
plt.tick_params(axis='both', which='major', labelsize=16)
plt.savefig("figs/figure_spectrum_legacysurvey_%s.png"%suffix)


In [ ]:
#here we prepare output. Images will not be forwarded in production, but now as it is
with open('data/catalog_legacysurvey_%s.ecsv'%suffix, 'r') as catfd, \
     open('data/spectrum_legacysurvey_%s.ecsv'%suffix, 'r') as specfd, \
     fits.open('data/image_legacysurvey_%s.fits'%suffix, 'readonly') as imfitsfd:
    catalog = catfd.read()
    spec = specfd.read()
    imfits_head = {k: v for k, v in imfitsfd[0].header.items()}
    imfits_data = imfitsfd[0].data

In [ ]:
catalog = catalog
spec = spec
imfits_head = imfits_head
imfits_data = imfits_data